In [1]:
import laspy
import pdal
from matplotlib import pyplot as plt
import numpy as np

# New imports needed for _safe_parse_crs
from pyproj import CRS
from pyproj.exceptions import CRSError
try:                                    # laspy ≥ 2.5
    from laspy.vlrs.known import GeoKeyDirectoryVLR
except ImportError:
    try:                                # laspy 2.0 – 2.4 (lower-case “r”)
        from laspy.vlrs.known import GeoKeyDirectoryVlr as GeoKeyDirectoryVLR
    except ImportError:                 # very old laspy (≤ 1.x)
        GeoKeyDirectoryVLR = None

# -------------------------------------------------
# Path + LAS read
# -------------------------------------------------
PATH = r"\\smb.isipd.dmawi.de\projects\legacy\stkruse\PermaX\2025_las_680\plot_06-10\FULL_ALS_L1B_20250809T172417_172709.las"
print(f"[main] LAS file path set to: {PATH}")

#print("[main] Reading LAS file with laspy.read() ...")
#las_file = laspy.read(PATH)  # <- only needed if you want point data; CRS can be read from header alone
#print("[main] LAS file read successfully.")

import laspy
from pyproj import CRS
from pyproj.exceptions import CRSError

# --- resolve GeoKeyDirectoryVLR for different laspy versions ---
try:                                    # laspy ≥ 2.5
    from laspy.vlrs.known import GeoKeyDirectoryVLR
except ImportError:
    try:                                # laspy 2.0 – 2.4
        from laspy.vlrs.known import GeoKeyDirectoryVlr as GeoKeyDirectoryVLR
    except ImportError:                 # very old laspy (≤ 1.x)
        GeoKeyDirectoryVLR = None

print(f"[crs] GeoKeyDirectoryVLR resolved as: {GeoKeyDirectoryVLR}")


def _decode_geotiff_key_list(key_list):
    """
    Convert GeoTIFF key info to {key_id: value}.

    Handles:
      - old style: flat list of uint16 (key_directory array)
      - new style: list of GeoKeyEntryStruct objects
    """
    print("[_decode_geotiff_key_list] Decoding raw GeoTIFF key list ...")

    if not key_list:
        print("[_decode_geotiff_key_list] Empty key list; returning empty dict.")
        return {}

    first = key_list[0]

    # --- for new laspy list of GeoKeyEntryStruct ---
    if hasattr(first, "key_id"):
        print("[_decode_geotiff_key_list] Detected list of GeoKeyEntryStruct entries.")
        out = {}
        for entry in key_list:
            # store only "inline" values (tiff_tag_location == 0, count == 1)
            if getattr(entry, "tiff_tag_location", None) == 0 and getattr(entry, "count", None) == 1:
                out[entry.key_id] = entry.value_offset
                print(
                    f"[_decode_geotiff_key_list] Inline key: "
                    f"key_id={entry.key_id}, value={entry.value_offset}"
                )
        print(f"[_decode_geotiff_key_list] Finished. {len(out)} keys extracted (GeoKeyEntryStruct).")
        return out

    # --- for old laspy flat list of uint16 ---
    if len(key_list) < 4:
        print("[_decode_geotiff_key_list] Key list too short; returning empty dict.")
        return {}

    num_keys = int(key_list[3])
    print(f"[_decode_geotiff_key_list] Detected flat uint16 array. Number of keys: {num_keys}")
    out = {}
    for i in range(num_keys):
        base = 4 + i * 4
        key_id, tag_loc, count, value = key_list[base: base + 4]
        if tag_loc == 0 and count == 1:   # value stored inline
            out[key_id] = value
            print(f"[_decode_geotiff_key_list] Inline key: key_id={key_id}, value={value}")

    print(f"[_decode_geotiff_key_list] Finished. {len(out)} keys extracted (uint16 array).")
    return out


def _safe_parse_crs(header):
    """
    Return a pyproj.CRS or None.
      1) Try laspy's header.parse_crs() (covers WKT + clean GeoTIFF).
      2) If that fails, manually walk GeoTIFF VLRs to recover EPSG.
      3) Extra heuristic: detect truncated WGS84 GEOGCS WKT and fall back to EPSG:4326.
    """
    print("[_safe_parse_crs] Starting CRS parsing")

    # 1) Direct attempt via header.parse_crs()
    try:
        print("[_safe_parse_crs] Trying header.parse_crs() ...")
        crs = header.parse_crs()
        # In your pyproj version there is no .is_empty, so just check truthiness
        if crs:
            print(f"[_safe_parse_crs] CRS from header.parse_crs(): {crs}")
            return crs
        else:
            print("[_safe_parse_crs] header.parse_crs() returned None.")
    except CRSError as e:
        msg = str(e)
        print(f"[_safe_parse_crs] header.parse_crs() raised CRSError: {msg}")

        # --- Heuristic: truncated WGS84 GEOGCS WKT like:
        # GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",... AUTHORITY["EPSG","432
        if 'GEOGCS["WGS 84"' in msg or 'DATUM["WGS_1984"' in msg:
            print("[_safe_parse_crs] Detected truncated WGS84 GEOGCS in error message; "
                  "falling back to EPSG:4326.")
            try:
                return CRS.from_epsg(4326)
            except CRSError as e2:
                print(f"[_safe_parse_crs] Fallback CRS.from_epsg(4326) failed: {e2}")
                # continue to GeoTIFF scan as last resort

    except Exception as e:
        print(f"[_safe_parse_crs] header.parse_crs() raised unexpected error: {e}")

    # 2) Manual scan of GeoTIFF keys in VLRs
    print("[_safe_parse_crs] Falling back to GeoTIFF VLR scan ...")

    for i, vlr in enumerate(header.vlrs):
        print(f"[_safe_parse_crs] Inspecting VLR #{i}: type={type(vlr)}")

        is_geotiff_vlr = (
            (GeoKeyDirectoryVLR and isinstance(vlr, GeoKeyDirectoryVLR)) or
            (
                getattr(vlr, "user_id", "") == "LASF_Projection"
                and getattr(vlr, "record_id", None) == 34735
            )
        )

        if not is_geotiff_vlr:
            print(f"[_safe_parse_crs] VLR #{i} is not a GeoKeyDirectory; skipping.")
            continue

        print(f"[_safe_parse_crs] VLR #{i} looks like a GeoKeyDirectoryVLR.")

        raw = getattr(vlr, "geo_keys", None)

        if isinstance(raw, dict):            # laspy ≥ 2.3
            print("[_safe_parse_crs] geo_keys is dict (laspy ≥ 2.3); using directly.")
            key_dict = raw
        elif isinstance(raw, list):          # list of GeoKeyEntryStruct or uint16s
            print("[_safe_parse_crs] geo_keys is list; decoding.")
            key_dict = _decode_geotiff_key_list(raw)
        else:                                # last-chance: raw directory
            print("[_safe_parse_crs] Using key_directory as fallback; decoding.")
            key_dict = _decode_geotiff_key_list(getattr(vlr, "key_directory", []))

        epsg = key_dict.get(3072) or key_dict.get(2048)  # PCS or GCS
        print(f"[_safe_parse_crs] EPSG candidate from keys: {epsg}")

        if epsg:
            try:
                crs = CRS.from_epsg(epsg)
                print(f"[_safe_parse_crs] Built CRS from EPSG:{epsg} -> {crs}")
                return crs
            except CRSError as e:
                print(f"[_safe_parse_crs] CRS.from_epsg({epsg}) failed with CRSError: {e}")
            except Exception as e:
                print(f"[_safe_parse_crs] CRS.from_epsg({epsg}) failed with unexpected error: {e}")
        else:
            print("[_safe_parse_crs] No EPSG (3072/2048) key found in this VLR.")

    print("[_safe_parse_crs] No valid CRS could be determined; returning None.")
    return None


# --- example usage for any LAS / LAZ file ---

def get_crs_from_las(path):
    """Convenience wrapper: open LAS/LAZ file, return (CRS, EPSG or None)."""
    print(f"[get_crs_from_las] Opening file: {path}")
    # Note: we only open the file to read the *header*, not the full point cloud
    with laspy.open(path) as f:
        header = f.header
        crs = _safe_parse_crs(header)
        epsg = crs.to_epsg() if crs else None
        print(f"[get_crs_from_las] Final CRS: {crs}")
        print(f"[get_crs_from_las] Final EPSG: {epsg}")
        return crs, epsg

# Example:
crs, epsg = get_crs_from_las(PATH)
print(f"[main] Retrieved CRS: {crs}")
print(f"[main] Retrieved EPSG: {epsg}")


[main] LAS file path set to: \\smb.isipd.dmawi.de\projects\legacy\stkruse\PermaX\2025_las_680\plot_06-10\FULL_ALS_L1B_20250809T172417_172709.las
[crs] GeoKeyDirectoryVLR resolved as: <class 'laspy.vlrs.known.GeoKeyDirectoryVlr'>
[get_crs_from_las] Opening file: \\smb.isipd.dmawi.de\projects\legacy\stkruse\PermaX\2025_las_680\plot_06-10\FULL_ALS_L1B_20250809T172417_172709.las
[_safe_parse_crs] Starting CRS parsing
[_safe_parse_crs] Trying header.parse_crs() ...
[_safe_parse_crs] header.parse_crs() raised CRSError: Invalid WKT string:       GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","432
[_safe_parse_crs] Detected truncated WGS84 GEOGCS in error message; falling back to EPSG:4326.
[get_crs_from_las] Final CRS: EPSG:4326
[get_crs_from_las] Final EPSG: 4326
[main] Retrieved CRS: EPSG:4326

In [2]:

#get crs from las file


import laspy

def dump_las_info(path):
    print(f"=== LAS INFO for: {path} ===")
    with laspy.open(path) as reader:
        hdr = reader.header

        print("\n--- BASIC HEADER ---")
        print(f"Version:            {hdr.version.major}.{hdr.version.minor}")
        # file_signature does not exist in laspy 2.x, so we skip it
        print(f"File Source ID:     {hdr.file_source_id}")
        print(f"Global Encoding:    {hdr.global_encoding}")
        #print(f"Project GUID:       {hdr.guid}")
        print(f"System Identifier:  {hdr.system_identifier}")
        print(f"Generating Software:{hdr.generating_software}")
        print(f"Creation Date:      {hdr.creation_date}")
        print(f"Point Count:        {hdr.point_count}")
        print(f"Point Format ID:    {hdr.point_format.id}")
        print(f"Point Record Length:{hdr.point_format.size}")

        print("\n--- SCALE & OFFSET ---")
        print(f"Scales:  {hdr.scales}")
        print(f"Offsets: {hdr.offsets}")

        print("\n--- BOUNDS (X/Y/Z) ---")
        print(f"Min: ({hdr.mins[0]}, {hdr.mins[1]}, {hdr.mins[2]})")
        print(f"Max: ({hdr.maxs[0]}, {hdr.maxs[1]}, {hdr.maxs[2]})")

        print("\n--- CRS ---")
        try:
            crs = hdr.parse_crs()
        except Exception as e:
            crs = None
            print(f"parse_crs() error: {e}")
        print(f"CRS (parse_crs): {crs}")

        print("\n--- VARIABLE LENGTH RECORDS (VLRs) ---")
        print(f"Number of VLRs: {len(hdr.vlrs)}")
        for i, vlr in enumerate(hdr.vlrs):
            print(f"  VLR #{i}:")
            print(f"    user_id:   {getattr(vlr, 'user_id', None)}")
            print(f"    record_id: {getattr(vlr, 'record_id', None)}")
            print(f"    description: {getattr(vlr, 'description', '')}")
            if getattr(vlr, 'user_id', '') == 'LASF_Projection':
                print(f"    (Projection-related VLR)")

        print("\n--- EXTENDED VARIABLE LENGTH RECORDS (EVLRs) ---")
        evlrs = getattr(hdr, "evlrs", [])
        print(f"Number of EVLRs: {len(evlrs)}")
        for i, evlr in enumerate(evlrs):
            print(f"  EVLR #{i}:")
            print(f"    user_id:   {getattr(evlr, 'user_id', None)}")
            print(f"    record_id: {getattr(evlr, 'record_id', None)}")
            print(f"    description: {getattr(evlr, 'description', '')}")

        print("\n--- POINT FORMAT / DIMENSIONS ---")
        pf = hdr.point_format
        print("Point format object:", pf)
        print("Dimensions:")
        for dim in pf:
            print(f"  - {dim.name} (type={dim.dtype})")

    print("=== END LAS INFO ===\n")

dump_las_info(PATH)



=== LAS INFO for: \\smb.isipd.dmawi.de\projects\legacy\stkruse\PermaX\2025_las_680\plot_06-10\FULL_ALS_L1B_20250809T172417_172709.las ===

--- BASIC HEADER ---
Version:            1.4
File Source ID:     0
Global Encoding:    <laspy.header.GlobalEncoding object at 0x000001E2F364A780>
System Identifier:  
Generating Software:LidarTools, IDL 8.5.1
Creation Date:      2025-08-09
Point Count:        20457893
Point Format ID:    6
Point Record Length:30

--- SCALE & OFFSET ---
Scales:  [1.e-07 1.e-07 1.e-02]
Offsets: [0. 0. 0.]

--- BOUNDS (X/Y/Z) ---
Min: (68.7731520099008, -133.47961217873998, -5.238901375792921)
Max: (-133.37754560435246, 111.89981115423143, 68.74175729096088)

--- CRS ---
parse_crs() error: Invalid WKT string:       GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","432
CRS (

In [3]:
import geopandas as gpd

gdf = gpd.read_file(r"\\smb.isipd.dmawi.de\projects\legacy\stkruse\PermaX\2025_las_680\plot_06-10\footprints\FULL_ALS_L1B_20250809T171641_172049.copc.gpkg")
gdf.explore()

In [ ]:
import numpy as np
import laspy
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import plotly.graph_objects as go
import ipywidgets as W
from pathlib import Path

# ---- User settings ----     # <- change this to your LAS/LAZ path
OUT_PATH = "your_file_cropped.las"  # output path
               # Plotly marker size
# ------------------------

def _safe_parse_crs(header):
    """
    Return a pyproj.CRS or None.
      1) Try laspy's header.parse_crs() (covers WKT + clean GeoTIFF).
      2) If that fails, manually walk GeoTIFF VLRs to recover EPSG.
    """
    try:
        crs = header.parse_crs()
        if crs and not crs.is_empty:
            return crs
    except CRSError:
        pass

    # Manual scan of GeoTIFF keys in VLRs
    for vlr in header.vlrs:
        if (GeoKeyDirectoryVLR and isinstance(vlr, GeoKeyDirectoryVLR)) or (
            getattr(vlr, "user_id", "") == "LASF_Projection" and getattr(vlr, "record_id", None) == 34735
        ):
            raw = getattr(vlr, "geo_keys", None)
            if isinstance(raw, dict):            # laspy ≥ 2.3
                key_dict = raw
            elif isinstance(raw, list):          # laspy ≤ 2.2
                key_dict = _decode_geotiff_key_list(raw)
            else:                                 # last-chance: raw directory
                key_dict = _decode_geotiff_key_list(getattr(vlr, "key_directory", []))

            epsg = key_dict.get(3072) or key_dict.get(2048)  # PCS or GCS
            if epsg:
                try:
                    return CRS.from_epsg(epsg)
                except CRSError:
                    pass

    return None

In [ ]:
las = laspy.read(LAS_PATH)
x_full = las.x.copy()
y_full = las.y.copy()
z_full = las.z.copy()

print(f"Loaded {len(x_full):,} points")

# Subsample for previews
idx_preview = np.arange(0, len(x_full), PREVIEW_STRIDE)
xp, yp, zp = x_full[idx_preview], y_full[idx_preview], z_full[idx_preview]

# Optional extra channels (if present)
intensity = getattr(las, "intensity", None)
intensity_p = intensity[idx_preview] if intensity is not None else None
classification = getattr(las, "classification", None)
class_p = classification[idx_preview] if classification is not None else None
